In [9]:
# neural turing machine with FF as controller
# see also https://github.com/flomlo/ntm_keras
import numpy as np
import math

def tanh_arr(x, d=False):
    if d:
        return 1 - np.tanh(x)
    else:
        return np.tanh(x)

def siginv_arr(y, d=False): # sigmoid inversed
    if d:
        return 1 / y / (1 - y)
    else:
        return np.log( y / (1 - y) )

def sig_arr(x, d=False): # sigmoid
    if d:
        return np.exp(-x) / (1 + np.exp(-x)) ** 2
    else:
        return 1 / (1 + np.exp(-x))

def softmax(x, d=False):
    ex = np.exp(x - np.max(x))
    res = ex / ex.sum()
    if d:
        return res * (1 - res)
    else:
        return res

def simK(u, v): # similarity measure
    return np.dot(u, v) / np.linalg.norm(u) / np.linalg.norm(v)

def weights(wt, wt_1, Mt, kt, betat, gt, st, gammat):
    nmemslots = np.shape(Mt)[0]
    for i in range(nmemslots):
        wt[i] = softmax(betat * simK(Mt[i], kt), False) # content addressing
    wt = gt * wt + (1 - gt) * wt_1 # gated weighing
    wt = np.convolve(wt, st, mode='same') # convolutional shift - todo: circular (does not make a big difference)
    wt = (np.sign(wt) * np.abs(wt) ** gammat) / np.sum(np.sign(wt) * np.abs(wt) ** gammat) # sharpening (careful fractional powers)
    return wt

def concat(atgt, *args): # custom concat
    res = atgt.flatten().tolist()
    for e in args:
        if type(e) is float:
            res += [e]
        elif type(e) is list:
            res += e
        else:
            res += e.flatten().tolist()
    return np.array(res)

def act_fwd(hc, csplit): # activation forward pass
    # subvectors: data + wt-1 + kt + beta + gt + st + gamma + et + at
    # lengths in csplit: ndata + nmemslots + m_depth + 1 + 1 + 3 + 1 + 2 * nmemslots
    # activations: ReLU + ReLU + tanh + ? + ? + soft + sig(inv, clip) + sig + tanh
    yc, wtc, ktc, betatc, gtc, stc, gammatc, etc, atc = np.split(hc, csplit)
    yc = np.maximum(yc, 0, yc) # ReLU
    wtc = np.maximum(wtc, 0, wtc) # ReLU
    ktc = tanh_arr(ktc, False)
    betatc = np.maximum(betatc, 0, betatc) # ReLU ?
    gtc = np.maximum(gtc, 0, gtc) # ReLU ?
    stc = softmax(stc, False)
    gammatc = np.clip(sig_arr(gammatc, False), -1, 1) # ?
    etc = sig_arr(etc, False)
    atc = tanh_arr(atc, False)
    return concat(yc, wtc, ktc, betatc, gtc, stc, gammatc, etc, atc)

def act_bck(hc, csplit): #  activation backpropagation
    # subvectors: data + wt-1 + kt + beta + gt + st + gamma + et + at
    # lengths in csplit: ndata + nmemslots + m_depth + 1 + 1 + 3 + 1 + 2 * nmemslots
    # activations: ReLU + ReLU + tanh + ? + ? + soft + sig(inv, clip) + sig + tanh
    yc, wtc, ktc, betatc, gtc, stc, gammatc, etc, atc = np.split(hc, csplit)
    yc = ((yc > 0) * 1.) # ReLU
    wtc = ((wtc > 0) * 1.) # ReLU
    ktc = tanh_arr(ktc, True)
    betatc = ((betatc > 0) * 1.) # ReLU ?
    gtc = ((gtc > 0) * 1.) # ReLU ?
    stc = softmax(stc, True)
    gammatc = np.clip(sig_arr(gammatc, True), -1, 1) # ?
    etc = sig_arr(etc, True)
    atc = tanh_arr(atc, True)
    return concat(yc, wtc, ktc, betatc, gtc, stc, gammatc, etc, atc)

if __name__=="__main__":
    # parameters
    ndata = 4 # size of external input data
    etha = 0.1 # learning rate
    ntmstps = 20
    
    # fill memory
    m_depth = 3
    Mt = np.array([[1, 0, 1], [0, 1, 1], [1, 1, 0], [1, 0, 1]]) # memory (N slots), m_depth=3
    nmemslots = np.shape(Mt)[0] # 4
    wt = np.array([0.25, 0.25, 0.25, 0.25]) # weights 1-N: one for each memory entry, initial values
    
    # the controller vectors are (data_vector, kt, beta, gt, st, gamma)
    kt = np.zeros((1, m_depth)) # key vector of length M (here: m_depth)
    betat = 0.8 # key strength
    gt = 0.5 # interpolation gate
    st = np.array([0.1, 0.8, 0.1]) # shift weighting
    gammat = 2.0 # sharpening
    # length of controller vector: data + wt-1 + kt + beta + gt + st + gamma + et + at
    nctrlvec = ndata + nmemslots + m_depth + 1 + 1 + 3 + 1 + 2 * nmemslots
    ctrl_inpv = np.zeros((1, nctrlvec)) # input controller
    
    # split vector data + wt-1 + kt + beta + gt + st + gamma
    ctrl_out_split = np.array([ndata, ndata+nmemslots, ndata+nmemslots+m_depth, ndata+nmemslots+m_depth+1, ndata+nmemslots+m_depth+2, ndata+nmemslots+m_depth+5, ndata+nmemslots+m_depth+6, ndata+nmemslots+m_depth+6+nmemslots])
    et = np.zeros((nmemslots)) # erase vector
    at = np.zeros((nmemslots)) # add vector
    
    x = np.zeros((ndata)) # input external data
    y = np.zeros((ndata)) # output external data
    
    # model parameters
    W1 = np.random.randn(nctrlvec, nctrlvec)*0.01 # input to hidden
    W2 = np.random.randn(nctrlvec, nctrlvec)*0.01 # hidden to output
    b1 = np.zeros((1, nctrlvec)) # inp-hidden bias
    b2 = np.zeros((1, nctrlvec)) # hidden-out bias
    
    # time steps
    for thist in range(ntmstps):

        # head moves todo: add multihead
        iheadpos = 0
        while iheadpos < nmemslots:
            wt_1 = wt

            # old memory Mt, read-weights wt from last step: feed read-vector to controller
            # calculate read vector to feed controller
            # read output vector from head location
            rt = np.dot(wt, Mt) # dim M, weighted memories

            # controller runs a single step (with input from outside)
            x = np.array([[0., 1., 0., 0.]]) # receive input vector from outside todo: really outside!
            ctrl_inpv = concat(x, wt, kt, betat, gt, st, gammat, et, at)
            # set indices for erase/add
            if iheadpos > 0:
                et[iheadpos-1] = 0
                at[iheadpos-1] = 0
            et[iheadpos] = 1
            at[iheadpos] = 1
            # controller forward pass, single step
            h1 = np.dot(ctrl_inpv, W1) + b1
            #h1 = np.maximum(h1, 0, h1) # ReLU todo: different by parameter
            h1 = act_fwd(h1[0], ctrl_out_split) # forward pass (different activation by subvector)
            o2 = np.dot(h1, W2) + b2
            # backward pass
            y[1] = 1. # arbitrary external output data (truth), todo: check, what is the entire truth?
            ctrl_outv = concat(y, wt, kt, betat, gt, st, gammat, et, at)
            #dW1 = - etha * (o2 - ctrl_outv) * np.maximum(h1, 0, h1)
            dW1 = - etha * (o2 - ctrl_outv) * h1
            dW2 = dW1 * act_bck(h1, ctrl_out_split) * ctrl_inpv
            W1 += dW1
            W2 += dW2

            # controller unactivated output divided into actual data output, reading, writing instructions
            # split: output_dim, read_heads, write_heads
            # split and apply activations:
            # k and add_vector are activated via tanh, erase_vector via sigmoid (this is critical!),
            # shift via softmax, gamma is sigmoided, inversed and clipped (probably not ideal)
            # g is sigmoided, beta is linear (probably not ideal!)
            y, wt, kt, betat, gt, st, gammat, et, at = np.split(o2[0], ctrl_out_split)

            # write memory for each head: calculate weights, erase, add
            wt = weights(wt, wt_1, Mt, kt, betat, gt, st, gammat)
            Mt = Mt - Mt * np.dot(wt, et.T) # erase
            Mt = Mt + np.dot(wt, at.T) # add

            # calculate read weights, save in the state and use for next round
            # todo
            iheadpos += 1
            
            print(Mt)

[[1.0056956  0.00283298 1.0056956 ]
 [0.00283298 1.0056956  1.0056956 ]
 [1.0056956  1.0056956  0.00283298]
 [1.0056956  0.00283298 1.0056956 ]]
[[ 1.00738054 -0.00445907  1.00738054]
 [-0.00445907  1.00738054  1.00738054]
 [ 1.00738054  1.00738054 -0.00445907]
 [ 1.00738054 -0.00445907  1.00738054]]
[[1.04046079 0.02812179 1.04046079]
 [0.02812179 1.04046079 1.04046079]
 [1.04046079 1.04046079 0.02812179]
 [1.04046079 0.02812179 1.04046079]]
[[1.11678237 0.0192001  1.11678237]
 [0.0192001  1.11678237 1.11678237]
 [1.11678237 1.11678237 0.0192001 ]
 [1.11678237 0.0192001  1.11678237]]
[[1.14458933 0.03843599 1.14458933]
 [0.03843599 1.14458933 1.14458933]
 [1.14458933 1.14458933 0.03843599]
 [1.14458933 0.03843599 1.14458933]]
[[ 0.33186761 -0.75729888  0.33186761]
 [-0.75729888  0.33186761  0.33186761]
 [ 0.33186761  0.33186761 -0.75729888]
 [ 0.33186761 -0.75729888  0.33186761]]
[[ 0.36044297 -0.71576451  0.36044297]
 [-0.71576451  0.36044297  0.36044297]
 [ 0.36044297  0.36044297 -0

In [13]:
import random

def rdbinseq(l): # random sequence of 8-bit binary vectors (int 0-255)
    if l <= 0:
        return []
    res = []
    for i in range(l):
        res.append(random.randint(0, 255))
    return res

def rdbinseqb(n, l): # random sequence of n-bit binary vectors
    if l <= 0:
        return []
    res = []
    for i in range(l):
        res.append([random.randint(0, 1) for j in range(n)])
    return res

if __name__=="__main__":
    print(int('11111111', 2))
    print(rdbinseq(10))
    print(rdbinseqb(8, 10))

255
[94, 49, 73, 128, 174, 45, 45, 198, 156, 2]
[[1, 1, 1, 0, 0, 1, 1, 0], [1, 1, 1, 0, 1, 1, 0, 0], [0, 1, 0, 0, 0, 1, 1, 1], [1, 1, 1, 0, 1, 1, 1, 1], [0, 1, 0, 1, 1, 1, 1, 1], [0, 0, 1, 0, 0, 1, 0, 0], [1, 1, 1, 0, 1, 1, 1, 0], [1, 0, 1, 1, 0, 0, 1, 0], [1, 0, 1, 1, 0, 1, 0, 1], [0, 0, 1, 1, 1, 1, 1, 0]]


In [20]:
# neural turing machine with FF as controller
# copy task
import numpy as np
import math

def rdbinseqb(n, l): # random sequence of n-bit binary vectors
    if l <= 0:
        return []
    res = []
    for i in range(l):
        res.append([random.randint(0, 1) for j in range(n)])
    return res

def tanh_arr(x, d=False):
    if d:
        return 1 - np.tanh(x)
    else:
        return np.tanh(x)

def siginv_arr(y, d=False): # sigmoid inversed
    if d:
        return 1 / y / (1 - y)
    else:
        return np.log( y / (1 - y) )

def sig_arr(x, d=False): # sigmoid
    if d:
        return np.exp(-x) / (1 + np.exp(-x)) ** 2
    else:
        return 1 / (1 + np.exp(-x))

def softmax(x, d=False):
    ex = np.exp(x - np.max(x))
    res = ex / ex.sum()
    if d:
        return res * (1 - res)
    else:
        return res

def simK(u, v): # similarity measure
    return np.dot(u, v) / np.linalg.norm(u) / np.linalg.norm(v)

def weights(wt, wt_1, Mt, kt, betat, gt, st, gammat):
    nmemslots = np.shape(Mt)[0]
    for i in range(nmemslots):
        wt[i] = softmax(betat * simK(Mt[i], kt), False) # content addressing
    wt = gt * wt + (1 - gt) * wt_1 # gated weighing
    wt = np.convolve(wt, st, mode='same') # convolutional shift - todo: circular (does not make a big difference)
    wt = (np.sign(wt) * np.abs(wt) ** gammat) / np.sum(np.sign(wt) * np.abs(wt) ** gammat) # sharpening (careful fractional powers)
    return wt

def concat(atgt, *args): # custom concat
    res = atgt.flatten().tolist()
    for e in args:
        if type(e) is float:
            res += [e]
        elif type(e) is list:
            res += e
        else:
            res += e.flatten().tolist()
    return np.array(res)

def act_fwd(hc, csplit): # activation forward pass
    # subvectors: data + wt-1 + kt + beta + gt + st + gamma + et + at
    # lengths in csplit: ndata + nmemslots + m_depth + 1 + 1 + 3 + 1 + 2 * nmemslots
    # activations: ReLU + ReLU + tanh + ? + ? + soft + sig(inv, clip) + sig + tanh
    yc, wtc, ktc, betatc, gtc, stc, gammatc, etc, atc = np.split(hc, csplit)
    yc = np.maximum(yc, 0, yc) # ReLU
    wtc = np.maximum(wtc, 0, wtc) # ReLU
    ktc = tanh_arr(ktc, False)
    betatc = np.maximum(betatc, 0, betatc) # ReLU ?
    gtc = np.maximum(gtc, 0, gtc) # ReLU ?
    stc = softmax(stc, False)
    gammatc = np.clip(sig_arr(gammatc, False), -1, 1) # ?
    etc = sig_arr(etc, False)
    atc = tanh_arr(atc, False)
    return concat(yc, wtc, ktc, betatc, gtc, stc, gammatc, etc, atc)

def act_bck(hc, csplit): #  activation backpropagation
    # subvectors: data + wt-1 + kt + beta + gt + st + gamma + et + at
    # lengths in csplit: ndata + nmemslots + m_depth + 1 + 1 + 3 + 1 + 2 * nmemslots
    # activations: ReLU + ReLU + tanh + ? + ? + soft + sig(inv, clip) + sig + tanh
    yc, wtc, ktc, betatc, gtc, stc, gammatc, etc, atc = np.split(hc, csplit)
    yc = ((yc > 0) * 1.) # ReLU
    wtc = ((wtc > 0) * 1.) # ReLU
    ktc = tanh_arr(ktc, True)
    betatc = ((betatc > 0) * 1.) # ReLU ?
    gtc = ((gtc > 0) * 1.) # ReLU ?
    stc = softmax(stc, True)
    gammatc = np.clip(sig_arr(gammatc, True), -1, 1) # ?
    etc = sig_arr(etc, True)
    atc = tanh_arr(atc, True)
    return concat(yc, wtc, ktc, betatc, gtc, stc, gammatc, etc, atc)

if __name__=="__main__":
    # parameters
    ndata = 4 # size of external input data
    etha = 0.1 # learning rate
    ntmstps = 20
    
    # fill memory
    m_depth = 3
    #Mt = np.array([[1, 0, 1], [0, 1, 1], [1, 1, 0], [1, 0, 1]]) # memory (N slots), m_depth=3
    Mt = np.array(rdbinseqb(3, 4))
    print(Mt)
    nmemslots = np.shape(Mt)[0] # 4
    wt = np.array([0.25, 0.25, 0.25, 0.25]) # weights 1-N: one for each memory entry, initial values
    
    # the controller vectors are (data_vector, kt, beta, gt, st, gamma)
    kt = np.zeros((1, m_depth)) # key vector of length M (here: m_depth)
    betat = 0.8 # key strength
    gt = 0.5 # interpolation gate
    st = np.array([0.1, 0.8, 0.1]) # shift weighting
    gammat = 2.0 # sharpening
    # length of controller vector: data + wt-1 + kt + beta + gt + st + gamma + et + at
    nctrlvec = ndata + nmemslots + m_depth + 1 + 1 + 3 + 1 + 2 * nmemslots
    ctrl_inpv = np.zeros((1, nctrlvec)) # input controller
    
    # split vector data + wt-1 + kt + beta + gt + st + gamma
    ctrl_out_split = np.array([ndata, ndata+nmemslots, ndata+nmemslots+m_depth, ndata+nmemslots+m_depth+1, ndata+nmemslots+m_depth+2, ndata+nmemslots+m_depth+5, ndata+nmemslots+m_depth+6, ndata+nmemslots+m_depth+6+nmemslots])
    et = np.zeros((nmemslots)) # erase vector
    at = np.zeros((nmemslots)) # add vector
    
    x = np.zeros((ndata)) # input external data
    y = np.zeros((ndata)) # output external data
    
    # model parameters
    W1 = np.random.randn(nctrlvec, nctrlvec)*0.01 # input to hidden
    W2 = np.random.randn(nctrlvec, nctrlvec)*0.01 # hidden to output
    b1 = np.zeros((1, nctrlvec)) # inp-hidden bias
    b2 = np.zeros((1, nctrlvec)) # hidden-out bias
    
    # time steps
    for thist in range(ntmstps):

        # head moves todo: add multihead
        iheadpos = 0
        while iheadpos < nmemslots:
            wt_1 = wt

            # old memory Mt, read-weights wt from last step: feed read-vector to controller
            # calculate read vector to feed controller
            # read output vector from head location
            rt = np.dot(wt, Mt) # dim M, weighted memories

            # controller runs a single step (with input from outside)
            x = np.array([[0., 1., 0., 0.]]) # receive input vector from outside todo: really outside!
            ctrl_inpv = concat(x, wt, kt, betat, gt, st, gammat, et, at)
            # set indices for erase/add
            if iheadpos > 0:
                et[iheadpos-1] = 0
                at[iheadpos-1] = 0
            et[iheadpos] = 1
            at[iheadpos] = 1
            # controller forward pass, single step
            h1 = np.dot(ctrl_inpv, W1) + b1
            #h1 = np.maximum(h1, 0, h1) # ReLU todo: different by parameter
            h1 = act_fwd(h1[0], ctrl_out_split) # forward pass (different activation by subvector)
            o2 = np.dot(h1, W2) + b2
            # backward pass
            y[1] = 1. # arbitrary external output data (truth), todo: check, what is the entire truth?
            ctrl_outv = concat(y, wt, kt, betat, gt, st, gammat, et, at)
            #dW1 = - etha * (o2 - ctrl_outv) * np.maximum(h1, 0, h1)
            dW1 = - etha * (o2 - ctrl_outv) * h1
            dW2 = dW1 * act_bck(h1, ctrl_out_split) * ctrl_inpv
            W1 += dW1
            W2 += dW2

            # controller unactivated output divided into actual data output, reading, writing instructions
            # split: output_dim, read_heads, write_heads
            # split and apply activations:
            # k and add_vector are activated via tanh, erase_vector via sigmoid (this is critical!),
            # shift via softmax, gamma is sigmoided, inversed and clipped (probably not ideal)
            # g is sigmoided, beta is linear (probably not ideal!)
            y, wt, kt, betat, gt, st, gammat, et, at = np.split(o2[0], ctrl_out_split)

            # write memory for each head: calculate weights, erase, add
            wt = weights(wt, wt_1, Mt, kt, betat, gt, st, gammat)
            Mt = Mt - Mt * np.dot(wt, et.T) # erase
            Mt = Mt + np.dot(wt, at.T) # add

            # calculate read weights, save in the state and use for next round
            # todo
            iheadpos += 1
            
            print(Mt)

[[0 0 1]
 [1 1 0]
 [0 1 1]
 [1 1 0]]
[[0.00282289 0.00282289 1.01358598]
 [1.01358598 1.01358598 0.00282289]
 [0.00282289 1.01358598 1.01358598]
 [1.01358598 1.01358598 0.00282289]]
[[0.00602052 0.00602052 1.0282275 ]
 [1.0282275  1.0282275  0.00602052]
 [0.00602052 1.0282275  1.0282275 ]
 [1.0282275  1.0282275  0.00602052]]
[[0.00931985 0.00931985 1.04338309]
 [1.04338309 1.04338309 0.00931985]
 [0.00931985 1.04338309 1.04338309]
 [1.04338309 1.04338309 0.00931985]]
[[0.01277678 0.01277678 1.05912542]
 [1.05912542 1.05912542 0.01277678]
 [0.01277678 1.05912542 1.05912542]
 [1.05912542 1.05912542 0.01277678]]
[[0.01602008 0.01602008 1.075015  ]
 [1.075015   1.075015   0.01602008]
 [0.01602008 1.075015   1.075015  ]
 [1.075015   1.075015   0.01602008]]
[[0.0194627  0.0194627  1.09139499]
 [1.09139499 1.09139499 0.0194627 ]
 [0.0194627  1.09139499 1.09139499]
 [1.09139499 1.09139499 0.0194627 ]]
[[0.02302059 0.02302059 1.10845757]
 [1.10845757 1.10845757 0.02302059]
 [0.02302059 1.108457